In [ ]:
import qmeq
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt

# Define an example system: Spinful single Anderson dot
- 2 levels
- Coulomb interaction
- 4 leads (2 on left and right for spin polarizations)

In [ ]:
n = 2
h = {(0,0):500, (1,1):500}
U = {(0,1,1,0):2000}

T = 100
nleads = 4
mulst = {0:0, 1:0, 2:0, 3:0}
tlst = {0:T, 1:2*T, 2:T, 3:2*T}

gammaL = 1.5
gammaR = .5
tL = np.sqrt(gammaL/np.pi/2)
tR = np.sqrt(gammaR/np.pi/2)

tleads = {(0, 0):tL, (1, 0):tR, (2, 1):tL, (3, 1):tR}


Define a system where we count at the left lead and use the pyLindblad approach, i.e. leads 0 and 2 with the way we chose couplings above:

In [ ]:

system_L = qmeq.Builder(nsingle=n, hsingle=h, coulomb=U, nleads=nleads,
                         mulst=mulst, tlst=tlst, tleads=tleads, dband=1e4, countingleads=[0,2], kerntype='pyLindblad')

Define a system where we count at the RIGHT lead and use the pyRTDnoise approach, i.e. leads 1 and 3 with the way we chose couplings above (note, that we set `off_diag_corrections=False`, they are not implemented):


In [ ]:

system_R = qmeq.Builder(nsingle=n, hsingle=h, coulomb=U, nleads=nleads,
                         mulst=mulst, tlst=tlst, tleads=tleads, dband=1e4, countingleads=[1,3], kerntype='pyRTDnoise')
system_R.off_diag_corrections=False

We can now as usual solve the system and also look at curren_noise.
Note,  that `system.current` is lead resolved, while `system.current_noise` contains currents and noise at a specified set of leads.

In [ ]:

system_L.solve()
system_R.solve()
print('Standard current system_L:',system_L.current)
print('Standard current system_R:',system_R.current)
print('Counting currents and noise at the left lead (Lindblad):',system_L.current_noise)
print('Counting currents and noise at the right lead (RTD):',system_R.current_noise.real)

Note here, that the current at the left lead is opposite the current at the right lead (current convention in QmeQ is that positive is from lead to dot)

We can like normal calculate for example gate sweeps:

In [ ]:
Vg = np.linspace(-1500, 3500, 100)

I = [[], []]
I_noise = [[], []]
S = [[], []]
for i,system in enumerate([system_L, system_R]):
    for vg in Vg:
        system.change(hsingle={(0,0):-vg, (1,1):-vg})
        system.solve()
        if system == system_L:
            I[i].append(system.current[0] + system.current[2])
        elif system == system_R:
            I[i].append(system.current[1] + system.current[3])
        I_noise[i].append(system.current_noise[0])
        S[i].append(system.current_noise[1])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].plot(Vg, I[0], label='L (std)', lw=3)
ax[0].plot(Vg, I[1], label='R (std)', lw=3)
ax[0].plot(Vg, I_noise[0], label='L (counting)')
ax[0].plot(Vg, I_noise[1], label='R (counting)')
ax[0].set_xlabel('$V_g$')
ax[0].set_ylabel('$I$')
ax[0].legend()
ax[1].plot(Vg, S[0], label='L')
ax[1].plot(Vg, S[1], label='R')
ax[1].set_xlabel('$V_g$')
ax[1].set_ylabel('$S$')
ax[1].legend()